# Index Vector Fields

本指南将引导您完成有关在集合中的向量字段上创建和管理索引的基本操作。

## 概述


利用存储在索引文件中的元数据，Milvus以专门的结构组织您的数据，便于在搜索或查询期间快速检索所需的信息。

Milvus提供了几种索引类型来对字段值进行排序，以实现高效的相似性搜索。它还提供了三种度量类型：余弦相似性（COSINE），欧几里得距离（L2）和内积（IP）来测量向量嵌入之间的距离。

建议为经常访问的向量字段和标量字段创建索引。

此页面上的代码片段使用新的MilvusClient（Python）与Milvus交互。其他语言的新MilvusClient SDK将在未来的更新中发布。

## Preparations

如管理集合中所述，如果在集合创建请求中指定了以下任何条件，则Milvus会在创建集合时自动生成索引并将其加载到内存中：

* 向量场的维数和度量类型，或
* 模式和索引参数。

下面的代码片段重新使用现有代码来建立到Milvus实例的连接，并创建一个集合，而无需指定其索引参数。在这种情况下，集合缺少索引，并保持卸载状态。


In [10]:
from pymilvus import MilvusClient, DataType

# 1. Set up a Milvus client
client = MilvusClient(
    uri="http://localhost:19530"
)

if client.has_collection("customized_setup"):
    client.drop_collection("customized_setup")

# 2. Create schema
# 2.1. Create schema
schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)

# 2.2. Add fields to schema
schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=5)

# 3. Create collection
client.create_collection(
    collection_name="customized_setup", 
    schema=schema, 
)

## Index a Collection

要为集合创建索引或索引集合，您需要设置索引参数并调用create_index()。

In [11]:
# 4.1. Set up the index parameters
index_params = MilvusClient.prepare_index_params()

# 4.2. Add an index on the vector field.
index_params.add_index(
    field_name="vector",
    metric_type="COSINE",
    index_type="IVF_FLAT",
    index_name="vector_index",
    params={"nlist": 1024}  # 添加 nlist 参数
)

# 4.3. Create an index file
client.create_index(
    collection_name="customized_setup",
    index_params=index_params
)

注意到

目前，只能为集合中的每个字段创建一个索引文件。

## Check Index Details

创建索引后，您可以查看其详细信息。

In [12]:
# 5. Describe index
res = client.list_indexes(
    collection_name="customized_setup"
)

print(res)

['vector_index']


In [13]:
res = client.describe_index(
    collection_name="customized_setup",
    index_name="vector_index"
)
print(res)

{'nlist': '1024', 'index_type': 'IVF_FLAT', 'metric_type': 'COSINE', 'field_name': 'vector', 'index_name': 'vector_index'}


您可以检查在特定字段上创建的索引文件，并收集有关使用此索引文件索引的行数的统计信息。

## 删除索引


如果不再需要索引，您可以简单地删除它。

In [14]:
# 6. Drop index
client.drop_index(
    collection_name="customized_setup",
    index_name="vector_index"
)

-----

# 索引标量字段

在Milvus中，标量索引用于通过特定的非向量字段值来加速元过滤，类似于传统的数据库索引。本指南将引导您为整数、字符串等字段创建和配置标量索引。


## 标量索引的类型


* 自动索引：Milvus根据标量字段的数据类型自动决定索引类型。这适用于不需要控制特定索引类型的情况。
* 自定义索引：指定确切的索引类型，如倒排索引.这提供了对索引类型选择的更多控制。



In [10]:
from pymilvus import MilvusClient, DataType

# 1. Set up a Milvus client
client = MilvusClient(
    uri="http://localhost:19530"
)

collection_name="test_scalar_index"

if client.has_collection(collection_name):
    client.drop_collection(collection_name)

# 2. Create schema
# 2.1. Create schema
schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)

# 2.2. Add fields to schema
schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="scalar_1", datatype=DataType.FLOAT_VECTOR, dim=5)
schema.add_field(field_name="scalar_2", datatype=DataType.INT8, dim=5)

# 3. Create collection
client.create_collection(
    collection_name=collection_name, 
    schema=schema, 
)

### 自动索引


要使用自动索引，请省略index_type参数，以便Milvus可以根据标量字段类型推断索引类型。有关标量数据类型和默认索引算法之间的映射，请参阅标量字段索引算法。


In [11]:
from pymilvus import MilvusClient

# Auto indexing
client = MilvusClient(uri="http://localhost:19530")
index_params = client.prepare_index_params() # Prepare an empty IndexParams object, without having to specify any index parameters

index_params.add_index(
    field_name="scalar_1", # Name of the scalar field to be indexed
    index_type="", # Type of index to be created. For auto indexing, leave it empty or omit this parameter.
    index_name="default_index" # Name of the index to be created
)

client.create_index(
  collection_name="test_scalar_index", # Specify the collection name
  index_params=index_params
)

### 定制索引化

若要使用自定义索引，请在index_type参数中指定特定的索引类型。


In [12]:
index_params = client.prepare_index_params() #  Prepare an IndexParams object

index_params.add_index(
    field_name="scalar_2", # Name of the scalar field to be indexed
    index_type="INVERTED", # Type of index to be created
    index_name="inverted_index" # Name of the index to be created
)

client.create_index(
  collection_name="test_scalar_index", # Specify the collection name
  index_params=index_params
)

**方法和参数**

- **创建索引参数**

  准备IndexParams对象。

- **add_index（）**

  将索引配置添加到IndexParams对象。

  - field_name（string）

    要建立索引的标量字段的名称。

  - index_type（string）：

    要创建的标量索引的类型。对于隐式索引，请将其保留为空或省略此参数。

    对于自定义索引，有效值为：

    - 倒排索引：（推荐）倒排索引由一个术语字典组成，其中包含按顺序排序的所有标记化的单词。有关详细信息，请参阅标量索引。
    - 使用标准模板库排序算法对标量字段进行排序。支持布尔和数值字段（例如，INT8、INT16、INT32、INT64、FLOAT、DOUBLE）。
    - Trie：用于快速前缀搜索和检索的树数据结构。支持VARCHAR字段。

  - index_name（string）

    要创建的标量索引的名称。每个标量字段支持一个索引。

- **创建索引（）**

  在指定集合中创建索引。

  - collection_name（string）

    为其创建索引的集合的名称。

  - **索引参数**

    包含索引配置的IndexParams对象。

## list_indexes

使用`list_indexes()`方法验证标量索引的创建：

In [13]:
client.list_indexes(
    collection_name="test_scalar_index"  # Specify the collection name
)

['default_index', 'inverted_index']

**目前，标量索引支持INT 8、INT 16、INT 32、INT 64、FLOAT、DOUBLE、BOOL和VARCHAR数据类型，但不支持JSON和ARRAY类型。**

--------

# 使用GPU索引

本指南概述了在Milvus中构建具有GPU支持的索引的步骤，这可以显著提高高吞吐量和高召回率场景中的搜索性能。有关Milvus支持的GPU索引类型的详细信息，请参阅GPU索引。


## 为GPU内存控制配置Milvus设置

Milvus使用全局图形内存池来分配GPU内存。

在Milvus配置文件中支持两个参数initMemSize和maxMemSize。池大小最初设置为initMemSize，超过此限制后将自动扩展到maxMemSize。

默认的initMemSize是Milvus启动时可用GPU内存的1/2，默认的maxMemSize等于所有可用GPU内存。


```bash
#when using GPU indexing, Milvus will utilize a memory pool to avoid frequent memory allocation and deallocation.
#here, you can set the size of the memory occupied by the memory pool, with the unit being MB.
#note that there is a possibility of Milvus crashing when the actual memory demand exceeds the value set by maxMemSize.
#if initMemSize and MaxMemSize both set zero,
#milvus will automatically initialize half of the available GPU memory,
#maxMemSize will the whole available GPU memory as default value.
gpu:
  initMemSize: 0 #set the initial memory pool size.
  maxMemSize: 0 #maxMemSize sets the maximum memory usage limit. When the memory usage exceed initMemSize, Milvus will attempt to expand the memory pool. 
```

## 建立索引

以下示例演示如何构建不同类型的GPU索引。

设置GPU索引参数时，请定义index_type、metric_type和params：

- index_type（string）：用于加速向量搜索的索引类型。有效选项包括GPU_CAGRA、GPU_IVF_FLAT、GPU_IVF_PQ和GPU_BRUTE_FORCE。
- metric_type（string）：用于度量向量相似性的度量类型。有效选项为IP和L2。
- params（dict）：索引特定的构建参数。此参数的有效选项取决于索引类型。

以下是不同索引类型的配置示例：

- GPU_CAGRA索引

```bash
index_params = {
    "metric_type": "L2",
    "index_type": "GPU_CAGRA",
    "params": {
        'intermediate_graph_degree': 64,
        'graph_degree': 32
    }
}
```

- 参数的可能选项包括：
  - intermediate_graph_degree（int）：通过在修剪之前确定图的度来影响召回率和构建时间。推荐值为32或64。
  - graph_degree（int）：通过设置修剪后图的度来影响搜索性能和召回率。通常，它是intermediate_graph_degree的一半。这两个度数之间的差异越大，构建时间就越长。它的值必须小于intermediate_graph_degree的值。
  - build_algo（string）：在剪枝之前验证图生成算法。可能的选择：
    - IVF_PQ：提供更高的质量，但更慢的构建时间。
    - NN_DESCENT：提供一个更快的构建，可能具有更低的召回率。
  - cache_dataset_on_device（string，“true”|“false”）：决定是否在GPU内存中缓存原始数据集。将其设置为“true”可以通过优化搜索结果来增强召回率，而将其设置为“false”则可以节省GPU内存。
- GPU_IVF_FLAT或GPU_IVF_PQ索引

```bash
index_params = {
    'index_type': 'GPU_BRUTE_FORCE',
    'metric_type': 'L2',
    'params': {}
}
```
不需要额外的参数配置。

## 建立索引

在index_params中配置索引参数后，调用create_index()方法构建索引。

In [ ]:
# Get an existing collection
collection = Collection("YOUR_COLLECTION_NAME")

collection.create_index(
    field_name="vector", # Name of the vector field on which an index is built
    index_params=index_params
)

## 搜索


构建GPU索引后，下一步是在执行搜索之前准备搜索参数。

准备搜索参数
以下是不同索引类型的示例配置：

- GPU_BRUTE_FORCE索引

```bash
search_params = {
    "metric_type": "L2",
    "params": {}
}
```

不需要额外的参数配置。

- GPU_CAGRA索引

```bash
search_params = {
    "metric_type": "L2",
    "params": {
        "itopk_size": 128,
        "search_width": 4,
        "min_iterations": 0,
        "max_iterations": 0,
        "team_size": 0
    }
}
```

- 关键搜索参数包括：
  - itopk_size：确定搜索过程中保留的中间结果的大小。较大的值可能会以牺牲搜索性能为代价来提高查全率。它应该至少等于最终的top-k（极限）值，并且通常是2的幂（例如，16、32、64、128）。
  - search_width：指定搜索过程中进入CAGRA图的入口点数。增加此值可以提高查全率，但可能会影响搜索性能。
  - min_iterations / max_iterations：这些参数控制搜索迭代过程。默认情况下，它们被设置为0，CAGRA会根据itopk_size和search_width自动确定迭代次数。手动调整这些值有助于平衡性能和准确性。
  - team_size：指定GPU上用于计算度量距离的CUDA线程数。常见的值是2到32的幂（例如2、4、8、16、32）。它对搜索性能有轻微影响。默认值为0，其中Milvus会根据向量维度自动选择team_size。
- GPU_IVF_FLAT或GPU_IVF_PQ索引

```bash
search_params = {
    "metric_type": "L2", 
    "params": {"nprobe": 10}
}
```

这两种索引类型的搜索参数类似于IVF_FLAT和IVF_PQ中使用的参数。有关详细信息，请参阅执行向量相似性搜索。

## 进行搜索

使用search()方法对GPU索引执行向量相似性搜索。

In [ ]:
# Load data into memory
collection.load()

collection.search(
    data=[[query_vector]], # Your query vector
    anns_field="vector", # Name of the vector field
    param=search_params,
    limit=100 # Number of the results to return
)

## 限制

使用GPU索引时，请注意某些限制：

- 对于GPU_IVF_FLAT，limit的最大值为256。
- 对于GPU_IVF_PQ和GPU_CAGRA，limit的最大值为1024。
- 虽然GPU_BRUTE_FORCE的限制没有设定限制，但建议不要超过4096，以避免潜在的性能问题。
- 目前，GPU索引不支持COSINE距离。如果需要COSINE距离，则应首先对数据进行归一化，然后可以使用内积（IP）距离作为替代。
- 不完全支持为GPU索引加载OOM保护，过多的数据可能会导致QueryNode崩溃。
- GPU索引不支持范围搜索和分组搜索等搜索功能。

## FAQ

- **什么时候应该使用GPU索引？**

  GPU索引在需要高吞吐量或高召回率的情况下特别有益。例如，在处理大批量时，GPU索引的吞吐量可以超过CPU索引的100倍。在小批量的场景中，GPU索引在性能方面仍然明显优于CPU索引。此外，如果需要快速插入数据，那么结合GPU可以大大加快构建索引的过程。

- **在哪些场景中，GPU索引（如CAGRA、GPU_IVF_PQ、GPU_IVF_FLAT和GPU_BRUTE_FORCE）最适合？**

  CAGRA索引非常适合需要增强性能的场景，尽管代价是消耗更多内存。对于优先考虑内存节约的环境，GPU_IVF_PQ索引可以帮助最大限度地减少存储需求，尽管这会带来更高的精度损失。GPU_IVF_FLAT索引作为一个平衡选项，在性能和内存使用之间提供折衷。最后，GPU_BRUTE_FORCE索引是为穷举搜索操作而设计的，通过执行遍历搜索来保证召回率为1。

-----